# Work on SED of CosmoDC2

- author : Sylvie Dagoret-Campagne
- Creation : January 29th 2021

In [1]:
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import pyarrow
import re # to decode SED

In [3]:
# Verify SparkContext
print(sc)

# Print Spark version
print(sc.version)

<SparkContext master=mesos://vm-75063.lal.in2p3.fr:5050 appName=PySparkShell>
2.4.4


In [4]:
# Spark imports
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F

In [5]:
from pyspark.sql.functions import col, pandas_udf 
from pyspark.sql.types import LongType

from pyspark.sql.types import IntegerType,FloatType
from pyspark.sql.functions import pandas_udf, PandasUDFType

from pyspark.sql import functions as F
from pyspark.sql.functions import randn

In [6]:
# here is how we create a function ('Ang2Pix') that can be called by dataframes
# it takes as input the "ra" and "dec" values (which are not very different from theta/phi)
# and returns the pixel number (but as pandas series for efficiency)
import numpy as np
import pandas as pd
import healpy as hp

nside=512

In [7]:
! hadoop dfs -ls /lsst/DC2/cosmoDC2

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Found 5 items
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-12-11 15:12 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-10-16 10:52 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_double.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-04-23 17:28 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_nofaint.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-09-07 19:17 /lsst/DC2/cosmoDC2/shear_cosmoDC2
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-04-16 18:40 /lsst/DC2/cosmoDC2/xyz_v1.1.4_hive


In [8]:
fullfilename="/lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image"
datafile=fullfilename
FLAG_ONE_FILE_INPUT=True

In [9]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

In [10]:
# Read the data as DataFrame
if FLAG_ONE_FILE_INPUT:
    df = spark.read.format("parquet").load(datafile)
else: # read all files
    #the_list_of_files=os.listdir(base_dir)
    #the_sorted_list_of_files=sorted(the_list_of_files)
    #print("sorted files : ",the_sorted_list_of_files[:5])
    #selected_files=[]
    #for filename in the_sorted_list_of_files:
    #    sel_filename=re.findall("^dc2_object_run2.2i_dr6c_tract[0-9][0-9][0-9][0-9].parquet$",filename)
    #    if len(sel_filename) > 0:
    #        selected_files.append(sel_filename[0])
    #print("selected files : ", selected_files[:5])
    #InputPath = [base_dir + "/{}".format(filename) for filename in selected_files]
    InputPath = all_files  # filename from the decoded index file
    df = spark.read.parquet(*InputPath)        

In [11]:
df.printSchema()

root
 |-- sed_2998_186_disk: float (nullable = true)
 |-- sed_17402_2596_disk: float (nullable = true)
 |-- sed_15143_2259_bulge: float (nullable = true)
 |-- Mag_true_y_lsst_z0: float (nullable = true)
 |-- sed_8329_517_bulge: float (nullable = true)
 |-- mag_i_lsst: double (nullable = true)
 |-- sed_13177_1966_disk: float (nullable = true)
 |-- mag_true_z_sdss: float (nullable = true)
 |-- sed_2407_591_no_host_extinction: float (nullable = true)
 |-- sed_1933_474_disk_no_host_extinction: float (nullable = true)
 |-- sed_3590_222: float (nullable = true)
 |-- sed_4048_251: float (nullable = true)
 |-- sed_8329_517: float (nullable = true)
 |-- mag_u_lsst_no_host_extinction: double (nullable = true)
 |-- velocity_x: double (nullable = true)
 |-- sed_1246_306_bulge: float (nullable = true)
 |-- sed_17402_2596_no_host_extinction: float (nullable = true)
 |-- sed_1552_381: float (nullable = true)
 |-- Mag_true_u_lsst_z0_no_host_extinction: float (nullable = true)
 |-- sed_6548_406_disk_no

In [12]:
list_of_columns=df.columns
print(list_of_columns)

['sed_2998_186_disk', 'sed_17402_2596_disk', 'sed_15143_2259_bulge', 'Mag_true_y_lsst_z0', 'sed_8329_517_bulge', 'mag_i_lsst', 'sed_13177_1966_disk', 'mag_true_z_sdss', 'sed_2407_591_no_host_extinction', 'sed_1933_474_disk_no_host_extinction', 'sed_3590_222', 'sed_4048_251', 'sed_8329_517', 'mag_u_lsst_no_host_extinction', 'velocity_x', 'sed_1246_306_bulge', 'sed_17402_2596_no_host_extinction', 'sed_1552_381', 'Mag_true_u_lsst_z0_no_host_extinction', 'sed_6548_406_disk_no_host_extinction', 'mag_true_z_lsst_no_host_extinction', 'sed_6954_431_bulge', 'mag_g_sdss_no_host_extinction', 'sed_4048_251_bulge', 'sed_3381_209_disk', 'position_y', 'sed_1552_381_disk_no_host_extinction', 'Mag_true_g_lsst_z0', 'Mag_true_i_lsst_z0', 'Mag_true_u_sdss_z0', 'sed_3184_197_bulge', 'galaxy_id', 'shear_2', 'mag_y_lsst', 'sed_5806_360_bulge_no_host_extinction', 'Mag_true_r_lsst_z0', 'Mag_true_z_lsst_z0', 'sed_1000_246_bulge_no_host_extinction', 'sed_4048_251_no_host_extinction', 'sed_6166_382_disk', 'sed_24

# Extract wavelengths

## Decode Wavelength bins

In [13]:
all_sed_columns = [ ]             # contain all names of columns with sed
all_sed_disk_columns = [ ]        # contain all names of columns with sed disk
all_sed_bulge_columns = [ ]        # contain all names of columns with sed bulge

all_sed_disk_no_host_extinction_columns = [ ]        # contain all names of columns with sed disk
all_sed_bulge_no_host_extinction_columns = [ ]        # contain all names of columns with sed bulge

all_disk = []                     # sed ending with disk
all_bulge = []                    # sed ending with bulge 
all_disk_no_host_extinction  = [] # sed ending with disk_no_host_extinction
all_bulge_no_host_extinction = [] # sed ending with disk_no_host_extinction

index=0
for x in list_of_columns:
    if re.match("^sed_*",x):
        all_sed_columns.append(x)
        
        # all_sed_disk_columns
        if re.match("^sed_.*_disk$",x):
            all_sed_disk_columns.append(x)  
        # all_sed_disk_columns
        if re.match("^sed_.*_bulge$",x):
            all_sed_bulge_columns.append(x)   
            
        # all_sed_disk_columns
        if re.match("^sed_.*_disk_no_host_extinction$",x):
            all_sed_disk_no_host_extinction_columns.append(x)  
        # all_sed_disk_columns
        if re.match("^sed_.*_bulge_no_host_extinction$",x):
            all_sed_bulge_no_host_extinction_columns.append(x)  
            
            
        # select only with _disk    
        nblist=re.findall('sed_([^ ]+)_disk$',x)
        if len(nblist)==1:
            all_disk.append(nblist[0])
        # select only with _bulge    
        nblist=re.findall('sed_([^ ]+)_bulge$',x)
        if len(nblist)==1:
            all_bulge.append(nblist[0])  
            
        # select only with _disk_no_host_extinction    
        nblist=re.findall('sed_([^ ]+)_disk_no_host_extinction$',x)
        if len(nblist)==1:
            all_disk_no_host_extinction.append(nblist[0])
        
        # select only with _bulge_no_host_extinction    
        nblist=re.findall('sed_([^ ]+)_bulge_no_host_extinction$',x)
        if len(nblist)==1:
            all_bulge_no_host_extinction.append(nblist[0])
                 
        index+=1
        


In [14]:
all_disk.sort()
all_bulge.sort()
all_disk_no_host_extinction.sort()
all_bulge_no_host_extinction.sort()

In [15]:
all_sed_disk_columns.sort()
all_sed_bulge_columns.sort()
all_sed_disk_no_host_extinction_columns.sort()
all_sed_bulge_no_host_extinction_columns.sort()

In [16]:
print(all_sed_disk_columns)

['sed_1000_246_disk', 'sed_11467_1710_disk', 'sed_1246_306_disk', 'sed_13177_1966_disk', 'sed_15143_2259_disk', 'sed_1552_381_disk', 'sed_17402_2596_disk', 'sed_1933_474_disk', 'sed_2407_591_disk', 'sed_2998_186_disk', 'sed_3184_197_disk', 'sed_3381_209_disk', 'sed_3590_222_disk', 'sed_3812_236_disk', 'sed_4048_251_disk', 'sed_4299_266_disk', 'sed_4565_283_disk', 'sed_4848_300_disk', 'sed_5148_319_disk', 'sed_5467_339_disk', 'sed_5806_360_disk', 'sed_6166_382_disk', 'sed_6548_406_disk', 'sed_6954_431_disk', 'sed_7385_458_disk', 'sed_7843_486_disk', 'sed_8329_517_disk', 'sed_8846_549_disk', 'sed_9395_583_disk', 'sed_9978_1489_disk']


In [17]:
print(all_sed_bulge_columns)

['sed_1000_246_bulge', 'sed_11467_1710_bulge', 'sed_1246_306_bulge', 'sed_13177_1966_bulge', 'sed_15143_2259_bulge', 'sed_1552_381_bulge', 'sed_17402_2596_bulge', 'sed_1933_474_bulge', 'sed_2407_591_bulge', 'sed_2998_186_bulge', 'sed_3184_197_bulge', 'sed_3381_209_bulge', 'sed_3590_222_bulge', 'sed_3812_236_bulge', 'sed_4048_251_bulge', 'sed_4299_266_bulge', 'sed_4565_283_bulge', 'sed_4848_300_bulge', 'sed_5148_319_bulge', 'sed_5467_339_bulge', 'sed_5806_360_bulge', 'sed_6166_382_bulge', 'sed_6548_406_bulge', 'sed_6954_431_bulge', 'sed_7385_458_bulge', 'sed_7843_486_bulge', 'sed_8329_517_bulge', 'sed_8846_549_bulge', 'sed_9395_583_bulge', 'sed_9978_1489_bulge']


In [18]:
print(all_sed_disk_no_host_extinction_columns)

['sed_1000_246_disk_no_host_extinction', 'sed_11467_1710_disk_no_host_extinction', 'sed_1246_306_disk_no_host_extinction', 'sed_13177_1966_disk_no_host_extinction', 'sed_15143_2259_disk_no_host_extinction', 'sed_1552_381_disk_no_host_extinction', 'sed_17402_2596_disk_no_host_extinction', 'sed_1933_474_disk_no_host_extinction', 'sed_2407_591_disk_no_host_extinction', 'sed_2998_186_disk_no_host_extinction', 'sed_3184_197_disk_no_host_extinction', 'sed_3381_209_disk_no_host_extinction', 'sed_3590_222_disk_no_host_extinction', 'sed_3812_236_disk_no_host_extinction', 'sed_4048_251_disk_no_host_extinction', 'sed_4299_266_disk_no_host_extinction', 'sed_4565_283_disk_no_host_extinction', 'sed_4848_300_disk_no_host_extinction', 'sed_5148_319_disk_no_host_extinction', 'sed_5467_339_disk_no_host_extinction', 'sed_5806_360_disk_no_host_extinction', 'sed_6166_382_disk_no_host_extinction', 'sed_6548_406_disk_no_host_extinction', 'sed_6954_431_disk_no_host_extinction', 'sed_7385_458_disk_no_host_exti

In [19]:
print(all_sed_bulge_no_host_extinction_columns)

['sed_1000_246_bulge_no_host_extinction', 'sed_11467_1710_bulge_no_host_extinction', 'sed_1246_306_bulge_no_host_extinction', 'sed_13177_1966_bulge_no_host_extinction', 'sed_15143_2259_bulge_no_host_extinction', 'sed_1552_381_bulge_no_host_extinction', 'sed_17402_2596_bulge_no_host_extinction', 'sed_1933_474_bulge_no_host_extinction', 'sed_2407_591_bulge_no_host_extinction', 'sed_2998_186_bulge_no_host_extinction', 'sed_3184_197_bulge_no_host_extinction', 'sed_3381_209_bulge_no_host_extinction', 'sed_3590_222_bulge_no_host_extinction', 'sed_3812_236_bulge_no_host_extinction', 'sed_4048_251_bulge_no_host_extinction', 'sed_4299_266_bulge_no_host_extinction', 'sed_4565_283_bulge_no_host_extinction', 'sed_4848_300_bulge_no_host_extinction', 'sed_5148_319_bulge_no_host_extinction', 'sed_5467_339_bulge_no_host_extinction', 'sed_5806_360_bulge_no_host_extinction', 'sed_6166_382_bulge_no_host_extinction', 'sed_6548_406_bulge_no_host_extinction', 'sed_6954_431_bulge_no_host_extinction', 'sed_73

In [20]:
print(all_disk)

['1000_246', '11467_1710', '1246_306', '13177_1966', '15143_2259', '1552_381', '17402_2596', '1933_474', '2407_591', '2998_186', '3184_197', '3381_209', '3590_222', '3812_236', '4048_251', '4299_266', '4565_283', '4848_300', '5148_319', '5467_339', '5806_360', '6166_382', '6548_406', '6954_431', '7385_458', '7843_486', '8329_517', '8846_549', '9395_583', '9978_1489']


In [21]:
print(all_bulge)

['1000_246', '11467_1710', '1246_306', '13177_1966', '15143_2259', '1552_381', '17402_2596', '1933_474', '2407_591', '2998_186', '3184_197', '3381_209', '3590_222', '3812_236', '4048_251', '4299_266', '4565_283', '4848_300', '5148_319', '5467_339', '5806_360', '6166_382', '6548_406', '6954_431', '7385_458', '7843_486', '8329_517', '8846_549', '9395_583', '9978_1489']


In [22]:
print(all_disk_no_host_extinction)

['1000_246', '11467_1710', '1246_306', '13177_1966', '15143_2259', '1552_381', '17402_2596', '1933_474', '2407_591', '2998_186', '3184_197', '3381_209', '3590_222', '3812_236', '4048_251', '4299_266', '4565_283', '4848_300', '5148_319', '5467_339', '5806_360', '6166_382', '6548_406', '6954_431', '7385_458', '7843_486', '8329_517', '8846_549', '9395_583', '9978_1489']


In [23]:
print(all_bulge_no_host_extinction)

['1000_246', '11467_1710', '1246_306', '13177_1966', '15143_2259', '1552_381', '17402_2596', '1933_474', '2407_591', '2998_186', '3184_197', '3381_209', '3590_222', '3812_236', '4048_251', '4299_266', '4565_283', '4848_300', '5148_319', '5467_339', '5806_360', '6166_382', '6548_406', '6954_431', '7385_458', '7843_486', '8329_517', '8846_549', '9395_583', '9978_1489']


In [24]:
def GetWavelength(wllist):
    """
    Decode wavelength from the string in list wllist
    """
    
    all_wl_aver =  []
    all_wl_width = []
    
    
    for wlstr in wllist:
        strarr= wlstr.split("_")
        
        wl=float(strarr[0])
        wdt=float(strarr[1])
        
        all_wl_aver.append(wl+wdt/2.)
        all_wl_width.append(wdt)
        
    return np.array(all_wl_aver),np.array(all_wl_width) 

In [25]:
# Decode wavelength
all_disk_wl_av , all_disl_wl_wdt = GetWavelength(all_disk)

In [26]:
print(all_disk_wl_av)

[ 1123.  12322.   1399.  14160.  16272.5  1742.5 18700.   2170.   2702.5
  3091.   3282.5  3485.5  3701.   3930.   4173.5  4432.   4706.5  4998.
  5307.5  5636.5  5986.   6357.   6751.   7169.5  7614.   8086.   8587.5
  9120.5  9686.5 10722.5]


In [27]:
wl_indexes=np.argsort(all_disk_wl_av)

In [28]:
wl_indexes

array([ 0,  2,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29,  1,  3,  4,  6])

In [29]:
all_disk_wl_av[wl_indexes]

array([ 1123. ,  1399. ,  1742.5,  2170. ,  2702.5,  3091. ,  3282.5,
        3485.5,  3701. ,  3930. ,  4173.5,  4432. ,  4706.5,  4998. ,
        5307.5,  5636.5,  5986. ,  6357. ,  6751. ,  7169.5,  7614. ,
        8086. ,  8587.5,  9120.5,  9686.5, 10722.5, 12322. , 14160. ,
       16272.5, 18700. ])

In [30]:
all_disl_wl_wdt[wl_indexes]

array([ 246.,  306.,  381.,  474.,  591.,  186.,  197.,  209.,  222.,
        236.,  251.,  266.,  283.,  300.,  319.,  339.,  360.,  382.,
        406.,  431.,  458.,  486.,  517.,  549.,  583., 1489., 1710.,
       1966., 2259., 2596.])

# Good columns selection

In [31]:
selected_cols_disk = ["galaxy_id","halo_id","position_x","position_y","position_z","redshift","ra","dec"] + all_sed_disk_columns

In [32]:
selected_cols_bulge = ["galaxy_id","halo_id","position_x","position_y","position_z","redshift","ra","dec"] + all_sed_bulge_columns

In [33]:
selected_cols_disk_no_host_extinction = ["galaxy_id","halo_id","position_x","position_y","position_z","redshift","ra","dec"] + all_sed_disk_no_host_extinction_columns

In [34]:
selected_cols_bulge_no_host_extinction = ["galaxy_id","halo_id","position_x","position_y","position_z","redshift","ra","dec"] + all_sed_bulge_no_host_extinction_columns

In [35]:
rs1=df.select(selected_cols_disk)
rs2=df.select(selected_cols_bulge)
rs3=df.select(selected_cols_disk_no_host_extinction)
rs4=df.select(selected_cols_bulge_no_host_extinction)

In [36]:
#rs.printSchema()

In [37]:
rs1.select('galaxy_id').show()

+----------+
| galaxy_id|
+----------+
|9437500000|
|9437500001|
|9437500002|
|9437500003|
|9437500004|
|9437500005|
|9437500006|
|9437500064|
|9437500065|
|9437500066|
|9437500067|
|9437500068|
|9437500069|
|9437500070|
|9437500071|
|9437500072|
|9437500073|
|9437500074|
|9437500075|
|9437500076|
+----------+
only showing top 20 rows



In [38]:
rs1.select(F.min("galaxy_id"), F.max("galaxy_id")).first()

Row(min(galaxy_id)=1250000000, max(galaxy_id)=12082707417)

In [40]:
rs1.count()

324194504

In [41]:
rs_sel = rs1.where(rs1["galaxy_id"] == 9437500000)

In [43]:
rs_sel.show()

+----------+-------+----------------+------------------+------------------+--------------------+-----------------+-----------------+-----------------+-------------------+-----------------+-------------------+-------------------+-----------------+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+
| galaxy_id|halo_id|      position_x|        position_y|        position_z|            redshift|               ra|              dec|sed_1000_246_disk|sed_11467_1710_disk|sed_1246_306_disk|sed_13177_1966_disk|sed_15143_2259_disk|sed_1552_381_disk|sed_17402_2596_disk|sed_1933_474_disk|sed_2407_591_disk|sed_2998_186_dis

In [ ]:
#stat = rs1.describe()
#stat.show()

In [ ]:
# Get min/max redshift values
m = rs1.select(F.min("redshift"), F.max("redshift")).first()
zmin, zmax = m[0], m[1]

# Number of bins, and spacings for our histogram
Nbins = 50
dz = (zmax - zmin) / Nbins
step = (zmax - zmin) / Nbins

rs=rs.withColumn("bin", ((rs["redshift"]-zmin)/dz).astype('int'))
rs.show(20)


In [ ]:
rs.describe(['redshift','bin']).show()

In [ ]:
# create the h dataframe using groupBy and count
h=rs.groupBy("bin").count()
# look at some values
h.show(10)

In [ ]:
#sort h (see sort or orderBy in DOC)
h=h.sort("bin")
#always check some values
h.show(5)

In [ ]:
# add to h a column (named "loc") giving the center of the bin
h=h.withColumn("loc", zmin+dz*h["bin"]+dz/2)
h.show(5)

In [ ]:
# convert h to a pandas object (see toPandas in DOC)
# you may drop the "bin" column first (see drop in DOC)
p=h.toPandas()
print(p.columns,"size={}".format(p.index.size))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.bar(p['loc'].values,p['count'].values,dz,color='white',edgecolor='black')
plt.xlabel("redshift");